In [849]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torch.nn.functional as F
import torchtext

#torch.backends.cudnn.deterministic = True

In [850]:
#pip install transformers[torch]

In [851]:
#pip install bnlp_toolkit

In [852]:
DATA_DIR = '/kaggle/input/sentnob-sentiment-analysis-in-noisy-bangla-texts/SentNoB Dataset/'

In [853]:
data = pd.read_csv(DATA_DIR+'Train.csv')
data.head()

,Data,Label
0,মুগ্ধ হয়ে গেলাম মামু. আর তোমায় কি কমু. বলো তোম...,1
1,এই কুত্তার বাচ্চাদের জন্য দেশটা আজ এমন অবস্তায়...,2
2,ভাই আপনার কথাই যাদু রয়েছে,1
3,উওরটা আমার অনেক ভাল লেগেছে,1
4,আমার নিজের গাড়ী নিয়ে কি সাজেক যেতে পারবো না ?...,0


In [854]:
valid_data = pd.read_csv(DATA_DIR+'Val.csv')
valid_data.head()

,Data,Label
0,আর আমার খুবেই আনন্দ লাকছে ভাইটি চাকরি পেয়েছে,1
1,ভাই আমাদের আগের মেয়র আনিচুল হক নাই যে আমাদের ক...,2
2,আমি মার্ক ওয়েন আর সনির বিশাল ভক্ত । একটা সময় ভ...,1
3,৩ মাস না যেতেই একেকজন ফুলে ফেপে আলুর দম,2
4,"বাংলাদেশের পুলিশ হলো নিরীহ মানুষের যম , আর অত্...",2


Label description
* 0 -> neutral
* 1 -> positive
* 2 -> negative

In [855]:
test_data = pd.read_csv(DATA_DIR+'Test.csv')
test_data.head()

,Data,Label
0,"স্বাস্থ্যবান হতে চাই , আমি বয়সের তুলনায় অনেক ব...",0
1,ভাইয়া নতুন ভিডিও আসে না কেন,0
2,সৌরভ গাঙ্গুলী ছাড়া দাদাগিরি কখনো জমে উঠত না,0
3,ক্রিকেট কে বাচাতে হলে পাপকে অতিশিগ্রিই তাকেও গ...,2
4,আমিতো সেই ঝালপ্রিয়ো মানুষ,1


In [856]:
data.columns = ['TEXT_COL', 'LABEL_COL']
valid_data.columns = ['TEXT_COL', 'LABEL_COL']
test_data.columns = ['TEXT_COL', 'LABEL_COL']

In [857]:
data.head()

,TEXT_COL,LABEL_COL
0,মুগ্ধ হয়ে গেলাম মামু. আর তোমায় কি কমু. বলো তোম...,1
1,এই কুত্তার বাচ্চাদের জন্য দেশটা আজ এমন অবস্তায়...,2
2,ভাই আপনার কথাই যাদু রয়েছে,1
3,উওরটা আমার অনেক ভাল লেগেছে,1
4,আমার নিজের গাড়ী নিয়ে কি সাজেক যেতে পারবো না ?...,0


In [858]:
data.shape

(12575, 2)

In [859]:
data['LABEL_COL'].value_counts()

1    5133
2    4548
0    2894
Name: LABEL_COL, dtype: int64

In [860]:
data['LABEL_COL'].nunique()

3

In [861]:
data.loc[0, 'TEXT_COL']

'মুগ্ধ হয়ে গেলাম মামু. আর তোমায় কি কমু. বলো তোমায় কোথায় পামু. আমি তোমার সাথে যাবু. গান কমু তোমার মতো গলি ঘুরমু.'

In [862]:
from bnlp import BasicTokenizer

# Preprocessing

In [863]:
def tokenizer(text):
    basic_t = BasicTokenizer()
    tokens = basic_t.tokenize(text)
    return tokens
    

In [864]:
from torch.utils.data import Dataset
class TextDataset(Dataset):
    def __init__(self, text, label=None):
        self.text = text
        self.label = label

    def __len__(self):
        return len(self.text)
    def __getitem__(self, idx):
        return self.text[idx], self.label[idx]

In [865]:
from torchtext.vocab import build_vocab_from_iterator

In [866]:
train_iter = TextDataset(data['TEXT_COL'].values, data['LABEL_COL'].values)

In [867]:
valid_iter = TextDataset(valid_data['TEXT_COL'].values, valid_data['LABEL_COL'].values)

In [868]:
test_iter = TextDataset(valid_data['TEXT_COL'].values, valid_data['LABEL_COL'].values)

In [869]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [870]:
text_pipeline = lambda x: vocab(tokenizer(x))


In [871]:
label_pipeline = lambda x: int(x)

In [872]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list.to(device), label_list.to(device), offsets.to(device)

#train_iter = AG_NEWS(split='train')
#dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [873]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [874]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class RNN(nn.Module):
    def __init__(self, input_dim, embed_dim, rnn_hidden_size,output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim,embed_dim, padding_idx=0)
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size)
        self.fc = nn.Linear(rnn_hidden_size, output_dim)

    def forward(self, text, lengths):
        lengths = torch.as_tensor(lengths, dtype=torch.int64, device='cpu')
        embedded = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(
        embedded, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True
        )
        out, (hidden, cell) = self.rnn(out)
        #hidden.squeeze_(0)
        out = out[-1,:, :]
        
        output = self.fc(out)
        return output

In [875]:
num_class = len(set([label for (text, label) in train_iter]))
vocab_size = len(vocab)
emsize = 128
#model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [876]:
rnn_hidden_size = 256
fc_hidden_size = 64
torch.manual_seed(1)
model = RNN(vocab_size, emsize, rnn_hidden_size, num_class).to(device)

In [877]:
num_class

3

In [878]:
vocab_size

24526

In [879]:
# for batch_idx, batch_data in enumerate(train_iter):
#     print(batch_idx, batch_data[0])
#     break

In [880]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, batch_data in enumerate(dataloader):
        optimizer.zero_grad()
        text = batch_data[0].to(device)
        label = batch_data[1].to(device)
        lengths = batch_data[2].to(device)
        predicted_label = model(text, lengths)
        
        #print(label)
        #print(predicted_label)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, batch_data in enumerate(dataloader):
            text = batch_data[0].to(device)
            label = batch_data[1].to(device)
            lengths = batch_data[2].to(device)
            predicted_label = model(text, lengths)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [881]:

from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 128 # batch size for training

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_dataset = to_map_style_dataset(train_iter)
valid_dataset = to_map_style_dataset(valid_iter)
test_dataset = to_map_style_dataset(test_iter)


train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

RuntimeError: Length of all samples has to be greater than 0, but found an element in 'lengths' that is <= 0

In [ ]:
for idx, batch in enumerate(train_dataloader):
    print(len(batch[2]))
    break

In [ ]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

In [ ]:
sentiment_label = {0: "neutral",
                 1: "positive",
                 2: "negative"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = test_data.loc[1, 'TEXT_COL']

model = model.to("cpu")
print(ex_text_str)

print("This is a %s text" % sentiment_label[predict(ex_text_str, text_pipeline)])